In [0]:
# !pip install pytorch_ssim

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


PIXEL-SHUFFLE ERROR

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-13-a510ddcf6ce0> in <module>()
     69         if torch.cuda.is_available():
     70             z = z.cuda()
---> 71         fake_img = netG(z)
     72 
     73         netD.zero_grad()

7 frames
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/pixelshuffle.py in forward(self, input)
     44     @weak_script_method
     45     def forward(self, input):
---> 46         return F.pixel_shuffle(input, self.upscale_factor)
     47 
     48     def extra_repr(self):

RuntimeError: c % upscale_factor_squared == 0 ASSERT FAILED at /pytorch/aten/src/ATen/native/PixelShuffle.cpp:24, please report a bug to PyTorch. pixel_shuffle expects input channel to be divisible by square of upscale_factor, but got input with sizes [64, 256, 29, 29], upscale_factor=3, and self.size(1)=256 is not divisible by 9

Examples of Pixel Shuffle being used : 
https://www.programcreek.com/python/example/107679/torch.nn.PixelShuffle

**Network modified to work for any upscale factor:**

 Instead of using multiple blocks of pixelshuffle(sacel_factor=2). I've used only one block with pixelshuffle(scale_factor=x), where x is whatever scale factor is required.
This does not work for fractional sacle values. 

Also edited the final conv layer to have 64 inputs and 3 outputs.

model

In [0]:
import math

import torch.nn.functional as F
from torch import nn

class pixel_shuffle(nn.Module):
    def __init__(self, scale_factor):
        super(pixel_shuffle, self).__init__()
        self.scale_factor = scale_factor

    def forward(self, input):
        scale_factor = self.scale_factor
        _, in_channels, in_height, in_width = input.shape

        in_channels = int(in_channels)
        in_height = int(in_height)
        in_width = int(in_width)

        out_channels = in_channels // (scale_factor * scale_factor)
        # out_channels = in_channels / int(scale_factor * scale_factor)
        # out_channels = 64
        out_height = in_height * scale_factor
        out_width = in_width * scale_factor

        if scale_factor >= 1:
            input_view = input.view([-1, out_channels, int(scale_factor), int(scale_factor), in_height, in_width])
            shuffle_out = input_view.permute(0, 1, 4, 2, 5, 3)

        return shuffle_out.contiguous().view([-1, out_channels, out_height, out_width])


class Generator(nn.Module):
    def __init__(self, scale_factor):

        super(Generator, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=9, padding=4),
            nn.PReLU()
        )
        self.block2 = ResidualBlock(64)
        self.block3 = ResidualBlock(64)
        self.block4 = ResidualBlock(64)
        self.block5 = ResidualBlock(64)
        self.block6 = ResidualBlock(64)
        self.block7 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1, groups=1),
            nn.BatchNorm2d(64)
            )       
        block8 = [UpsampleBLock(64, scale_factor)]
        block8.append(nn.Conv2d( 64 , 3, kernel_size=9, padding=4, groups=1))

        self.block8 = nn.Sequential(*block8)

    def forward(self, x):
        block1 = self.block1(x)
        block2 = self.block2(block1)
        block3 = self.block3(block2)
        block4 = self.block4(block3)
        block5 = self.block5(block4)
        block6 = self.block6(block5)
        block7 = self.block7(block6)
        block8 = self.block8(block1 + block7)

        return (F.tanh(block8) + 1) / 2



class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(512, 1024, kernel_size=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(1024, 1, kernel_size=1)
        )

    def forward(self, x):
        batch_size = x.size(0)
        return F.sigmoid(self.net(x).view(batch_size))


class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.prelu = nn.PReLU()
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self, x):
        residual = self.conv1(x)
        residual = self.bn1(residual)
        residual = self.prelu(residual)
        residual = self.conv2(residual)
        residual = self.bn2(residual)

        return x + residual


class UpsampleBLock(nn.Module):
    def __init__(self, in_channels, up_scale):
        super(UpsampleBLock, self).__init__()
        self.conv = nn.Conv2d(in_channels, int(in_channels*up_scale ** 2), kernel_size=3, padding=1, groups=1)
        # self.pixel_shuffle = nn.PixelShuffle(up_scale)  
        self.pixel_shuffle = pixel_shuffle(up_scale)
        self.prelu = nn.PReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.pixel_shuffle(x)
        x = self.prelu(x)
        return x

utils


In [0]:
from os import listdir
from os.path import join

from PIL import Image
from torch.utils.data.dataset import Dataset
from torchvision.transforms import Compose, RandomCrop, ToTensor, ToPILImage, CenterCrop, Resize


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG'])


def calculate_valid_crop_size(crop_size, upscale_factor):
    return crop_size - (crop_size % int(upscale_factor))


def train_hr_transform(crop_size):
    return Compose([
        RandomCrop(crop_size),
        ToTensor(),
    ])


def train_lr_transform(crop_size, upscale_factor):
    return Compose([
        ToPILImage(),
        Resize(crop_size // int(upscale_factor), interpolation=Image.BICUBIC),
        ToTensor()
    ])


def display_transform():
    return Compose([
        ToPILImage(),
        Resize(400),
        CenterCrop(400),
        ToTensor()
    ])


class TrainDatasetFromFolder(Dataset):
    def __init__(self, dataset_dir, crop_size, upscale_factor):
        super(TrainDatasetFromFolder, self).__init__()
        self.image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir) if is_image_file(x)]
        crop_size = calculate_valid_crop_size(crop_size, upscale_factor)
        self.hr_transform = train_hr_transform(crop_size)
        self.lr_transform = train_lr_transform(crop_size, upscale_factor)

    def __getitem__(self, index):
        hr_image = self.hr_transform(Image.open(self.image_filenames[index]))
        lr_image = self.lr_transform(hr_image)
        return lr_image, hr_image

    def __len__(self):
        return len(self.image_filenames)


class ValDatasetFromFolder(Dataset):
    def __init__(self, dataset_dir, upscale_factor):
        super(ValDatasetFromFolder, self).__init__()
        self.upscale_factor = upscale_factor
        self.image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir) if is_image_file(x)]

    def __getitem__(self, index):
        hr_image = Image.open(self.image_filenames[index])
        w, h = hr_image.size
        crop_size = calculate_valid_crop_size(min(w, h), self.upscale_factor)
        lr_scale = Resize(crop_size // self.upscale_factor, interpolation=Image.BICUBIC)
        hr_scale = Resize(crop_size, interpolation=Image.BICUBIC)
        hr_image = CenterCrop(crop_size)(hr_image)
        lr_image = lr_scale(hr_image)
        hr_restore_img = hr_scale(lr_image)
        return ToTensor()(lr_image), ToTensor()(hr_restore_img), ToTensor()(hr_image)

    def __len__(self):
        return len(self.image_filenames)


class TestDatasetFromFolder(Dataset):
    def __init__(self, dataset_dir, upscale_factor):
        super(TestDatasetFromFolder, self).__init__()
        # self.lr_path = dataset_dir + '/SRF_' + str(upscale_factor) + '/data/'
        self.lr_path = dataset_dir + '/set5' + '/data'
        # self.hr_path = dataset_dir + '/SRF_' + str(upscale_factor) + '/target/'
        self.hr_path = dataset_dir + '/set5' + '/target'
        self.upscale_factor = upscale_factor
        self.lr_filenames = [join(self.lr_path, x) for x in listdir(self.lr_path) if is_image_file(x)]
        self.hr_filenames = [join(self.hr_path, x) for x in listdir(self.hr_path) if is_image_file(x)]

    def __getitem__(self, index):
        image_name = self.lr_filenames[index].split('/')[-1]
        lr_image = Image.open(self.lr_filenames[index])
        w, h = lr_image.size
        hr_image = Image.open(self.hr_filenames[index])
        hr_scale = Resize((self.upscale_factor * h, self.upscale_factor * w), interpolation=Image.BICUBIC)
        hr_restore_img = hr_scale(lr_image)
        return image_name, ToTensor()(lr_image), ToTensor()(hr_restore_img), ToTensor()(hr_image)

    def __len__(self):
        return len(self.lr_filenames)

loss

In [0]:
import torch
from torch import nn
from torchvision.models.vgg import vgg19


class GeneratorLoss(nn.Module):
    def __init__(self):
        super(GeneratorLoss, self).__init__()
        vgg = vgg19(pretrained=True)
        loss_network = nn.Sequential(*list(vgg.features)[:31]).eval()
        for param in loss_network.parameters():
            param.requires_grad = False
        self.loss_network = loss_network
        self.mse_loss = nn.MSELoss()
        self.tv_loss = TVLoss()

    def forward(self, out_labels, out_images, target_images):
        # Adversarial Loss
        adversarial_loss = torch.mean(1 - out_labels)
        # Perception Loss
        perception_loss = self.mse_loss(self.loss_network(out_images), self.loss_network(target_images))
        # Image Loss
        image_loss = self.mse_loss(out_images, target_images)
        # TV Loss
        tv_loss = self.tv_loss(out_images)
        return image_loss + 0.001 * adversarial_loss + 0.006 * perception_loss + 2e-8 * tv_loss


class TVLoss(nn.Module):
    def __init__(self, tv_loss_weight=1):
        super(TVLoss, self).__init__()
        self.tv_loss_weight = tv_loss_weight

    def forward(self, x):
        batch_size = x.size()[0]
        h_x = x.size()[2]
        w_x = x.size()[3]
        count_h = self.tensor_size(x[:, :, 1:, :])
        count_w = self.tensor_size(x[:, :, :, 1:])
        h_tv = torch.pow((x[:, :, 1:, :] - x[:, :, :h_x - 1, :]), 2).sum()
        w_tv = torch.pow((x[:, :, :, 1:] - x[:, :, :, :w_x - 1]), 2).sum()
        return self.tv_loss_weight * 2 * (h_tv / count_h + w_tv / count_w) / batch_size

    @staticmethod
    def tensor_size(t):
        return t.size()[1] * t.size()[2] * t.size()[3]


if __name__ == "__main__":
    g_loss = GeneratorLoss()
    print(g_loss)

train


In [0]:
import os
from math import log10

import pandas as pd
import torch.optim as optim
import torch.utils.data
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm

CROP_SIZE = 88 
UPSCALE_FACTOR = 4 
NUM_EPOCHS = 1000 

train_set = TrainDatasetFromFolder('/content/drive/My Drive/vooc2012/train', crop_size=CROP_SIZE, upscale_factor=UPSCALE_FACTOR)
val_set = ValDatasetFromFolder('/content/drive/My Drive/vooc2012/val', upscale_factor=UPSCALE_FACTOR)
train_loader = DataLoader(dataset=train_set, num_workers=4, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_set, num_workers=4, batch_size=1, shuffle=False)

netG = Generator(UPSCALE_FACTOR)
print('# generator parameters:', sum(param.numel() for param in netG.parameters()))
netD = Discriminator()
print('# discriminator parameters:', sum(param.numel() for param in netD.parameters()))

generator_criterion = GeneratorLoss()

if torch.cuda.is_available():
    netG.cuda()
    netD.cuda()
    generator_criterion.cuda()

optimizerG = optim.Adam(netG.parameters())
optimizerD = optim.Adam(netD.parameters())

results = {'d_loss': [], 'g_loss': [], 'd_score': [], 'g_score': [], 'psnr': [], 'ssim': []}

for epoch in range(1, NUM_EPOCHS + 1):
    train_bar = tqdm(train_loader)
    running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 'd_score': 0, 'g_score': 0}

    netG.train()
    netD.train()
    for data, target in train_bar:
        g_update_first = True
        batch_size = data.size(0)
        running_results['batch_sizes'] += batch_size

        ############################
        # (1) Update D network: maximize D(x)-1-D(G(z))
        ###########################
        real_img = Variable(target)
        if torch.cuda.is_available():
            real_img = real_img.cuda()
        z = Variable(data)
        if torch.cuda.is_available():
            z = z.cuda()
        fake_img = netG(z)
        
        netD.zero_grad()
        real_out = netD(real_img).mean()
        fake_out = netD(fake_img).mean()
        d_loss = 1 - real_out + fake_out
        d_loss.backward(retain_graph=True)
        optimizerD.step()

        ############################
        # (2) Update G network: minimize 1-D(G(z)) + Perception Loss + Image Loss + TV Loss
        ###########################
        netG.zero_grad()
        g_loss = generator_criterion(fake_out, fake_img, real_img)
        g_loss.backward()
        optimizerG.step()
        fake_img = netG(z)
        fake_out = netD(fake_img).mean()

        g_loss = generator_criterion(fake_out, fake_img, real_img)
        running_results['g_loss'] += g_loss.data * batch_size
        d_loss = 1 - real_out + fake_out
        running_results['d_loss'] += d_loss.data * batch_size
        running_results['d_score'] += real_out.data * batch_size
        running_results['g_score'] += fake_out.data * batch_size

        train_bar.set_description(desc='[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f batch_size: %.4f' % (
            epoch, NUM_EPOCHS, running_results['d_loss'] / running_results['batch_sizes'],
            running_results['g_loss'] / running_results['batch_sizes'],
            running_results['d_score'] / running_results['batch_sizes'],
            running_results['g_score'] / running_results['batch_sizes'],
            running_results['batch_sizes']))
    
    print('-----------------')
    print('LR image size:{}\n'.format(list(z.size())))
    print('SR image size:{}\n'.format(list(fake_img.size())))
    print('-----------------')

    netG.eval()
    out_path = 'training_results/SRF_' + str(UPSCALE_FACTOR) + '/'
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    val_bar = tqdm(val_loader)
    valing_results = {'mse': 0, 'ssims': 0, 'psnr': 0, 'ssim': 0, 'batch_sizes': 0}
    val_images = []
    for val_lr, val_hr_restore, val_hr in val_bar:
        batch_size = val_lr.size(0)
        valing_results['batch_sizes'] += batch_size
        lr = Variable(val_lr, volatile=True)
        hr = Variable(val_hr, volatile=True)
        if torch.cuda.is_available():
            lr = lr.cuda()
            hr = hr.cuda()
        sr = netG(lr)

        batch_mse = ((sr - hr) ** 2).data.mean()
        valing_results['mse'] += batch_mse * batch_size
        valing_results['psnr'] = 10 * log10(1 / (valing_results['mse'] / valing_results['batch_sizes']))
        val_bar.set_description(
            desc='[converting LR images to SR images] PSNR: %.4f dB ' % (
                valing_results['psnr']))

        val_images.extend(
            [display_transform()(val_hr_restore.squeeze(0)), display_transform()(hr.data.cpu().squeeze(0)),
             display_transform()(sr.data.cpu().squeeze(0))])
    val_images = torch.stack(val_images)
    val_images = torch.chunk(val_images, val_images.size(0) // 15)
    val_save_bar = tqdm(val_images, desc='[saving training results]')
    index = 1
    for image in val_save_bar:
        image = utils.make_grid(image, nrow=3, padding=5)
        utils.save_image(image, out_path + 'epoch_%d_index_%d.png' % (epoch, index), padding=5)
        index += 1

    torch.save(netG.state_dict(), 'netG_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch))
    torch.save(netD.state_dict(), 'netD_epoch_%d_%d.pth' % (UPSCALE_FACTOR, epoch))
    # save loss\scores\psnr\ssim
    results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
    results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
    results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
    results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])
    results['psnr'].append(valing_results['psnr'])

    if epoch % 10 == 0 and epoch != 0:  
        # out_path = 'statistics/'
        data_frame = pd.DataFrame( data={'Loss_D': results['d_loss'],
                                         'Loss_G': results['g_loss'],
                                         'Score_D': results['d_score'],
                                         'Score_G': results['g_score'],
                                         'PSNR': results['psnr'],
                                         },
                                  index=list(range(1, epoch + 1))
                                  )
        data_frame.to_csv( 'srf_' + str(UPSCALE_FACTOR) + '_train_results.csv', index_label='Epoch')

        # out_path
        # 'SSIM': results['ssim']   

# generator parameters: 669594
# discriminator parameters: 5215425

  0%|          | 0/17 [00:00<?, ?it/s]
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-18-8a6e2b51ad98> in <module>()
     69         if torch.cuda.is_available():
     70             z = z.cuda()
---> 71         fake_img = netG(z)
     72 
     73         netD.zero_grad()

7 frames
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/pixelshuffle.py in forward(self, input)
     44     @weak_script_method
     45     def forward(self, input):
---> 46         return F.pixel_shuffle(input, self.upscale_factor)
     47 
     48     def extra_repr(self):

TypeError: pixel_shuffle(): argument 'upscale_factor' (position 2) must be int, not float

single image

In [0]:
# import argparse
import time

import torch
from PIL import Image
from torch.autograd import Variable
from torchvision.transforms import ToTensor, ToPILImage

UPSCALE_FACTOR = 4 
TEST_MODE = True #True if opt.test_mode == 'GPU' else False
IMAGE_NAME = '0016x4.png'
MODEL_NAME = 'netG_epoch_4_931.pth'  

model = Generator(UPSCALE_FACTOR).eval()
if TEST_MODE:
    model.cuda()
    model.load_state_dict(torch.load( MODEL_NAME)) 
else:
    model.load_state_dict(torch.load( MODEL_NAME, map_location=lambda storage, loc: storage)) 

image = Image.open(IMAGE_NAME)
image = Variable(ToTensor()(image), volatile=True).unsqueeze(0)
if TEST_MODE:
    image = image.cuda()

start = time.clock()
out = model(image)
elapsed = (time.clock() - start)
print('cost' + str(elapsed) + 's')
out_img = ToPILImage()(out[0].data.cpu())
out_img.save('out_srf_' + str(UPSCALE_FACTOR) + '_' + IMAGE_NAME)

Single Image with PSNR calculation

In [0]:
import time

import torch
from PIL import Image
from torch.autograd import Variable
from torchvision.transforms import ToTensor, ToPILImage
from math import log10


UPSCALE_FACTOR = 4
TEST_MODE = True #True if opt.test_mode == 'GPU' else False
IMAGE_NAME = '22-4x.jpg'
MODEL_NAME = 'netG_epoch_4_931.pth' 
hr_image = '11-4x.jpg'

model = Generator(UPSCALE_FACTOR).eval()
if TEST_MODE:
    model.cuda()
    model.load_state_dict(torch.load( MODEL_NAME))
else:
    model.load_state_dict(torch.load( MODEL_NAME, map_location=lambda storage, loc: storage)) 

image = Image.open(IMAGE_NAME)
hr = Image.open(hr_image)

image = Variable(ToTensor()(image), volatile=True).unsqueeze(0)
hr = Variable(ToTensor()(hr), volatile=True).unsqueeze(0)

if TEST_MODE:
    image = image.cuda()
    hr = hr.cuda()

start = time.clock()
out = model(image)
elapsed = (time.clock() - start)
print('cost for {}:{}s\n'.format(IMAGE_NAME,str(elapsed)))

# print('cost' + str(elapsed) + 's')
mse = ((hr - out) ** 2).data.mean()
psnr = 10 * log10(1 / mse)
print('PSNR value for {}:{}\n'.format(IMAGE_NAME,psnr))
out_img = ToPILImage()(out[0].data.cpu())
out_img.save('out_srf_' + str(UPSCALE_FACTOR) + '_' + IMAGE_NAME)

In [0]:
torch.cuda.get_device_name(torch.cuda.current_device())

In [0]:
# hr_image = 'baby_rlt.png'
# hr = Image.open(hr_image)
# hr = Variable(ToTensor()(hr), volatile=True)

# mse = ((hr - out_t) ** 2).data.mean()
# psnr = 10 * log10(1 / mse)
# print(psnr)

In [0]:
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np

# data = pd.read_csv('srf_5_train_results111.csv')
# data.head()

In [0]:
# epochs = data.iloc[:,0]
# loss_d = data['Loss_D']
# # loss_d.head()

# loss_g = data['Loss_G']
# # loss_g.head()
# epochs.head()

In [0]:
# plt.plot(epochs, loss_g)
# plt.ylabel('Generator Loss')
# plt.xlabel('Epochs')
# plt.savefig('Generator_loss_100epx5')
# plt.show()

In [0]:
# plt.plot(epochs, loss_d)
# plt.ylabel('Discriminator Loss')
# plt.xlabel('Epochs')
# plt.savefig('Discriminator_loss_100epx5')
# plt.show()